观察到数据中有多个学院，根据学院分组，思政课各项指标，专业课各项指标，体育教育各项指标和美育教育劳动教育各项指标画出对应柱状图，为了方便对比，直接从高到低排序

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 读取数据
df = pd.read_csv('../../data/intermediate/初步清洗_比赛数据.csv', sep=',', engine='python')
# 处理制表符
df = df.applymap(lambda x: x.strip('\t') if isinstance(x, str) else x)


# 按学院分组计算平均满意度
categories = {
    '思政课': ['思政课总体满意度', '思政课设置满意度', '思政课内容满意度', '思政课质量满意度', '思政课效果满意度'],
    '专业课': ['专业课知识融合', '专业课解决问题能力', '专业课交叉融合', '专业课实践结合', '专业课努力程度', '专业课前沿内容'],
}

# 确保列名存在
missing_cols = [col for col in sum(categories.values(), []) if col not in df.columns]
if missing_cols:
    raise ValueError(f"缺失列：{missing_cols}")

# 计算各学院平均分
college_means = []
for name, group in df.groupby('学院'):
    college_means.append({
        '学院': name,
        '思政课': group[categories['思政课']].mean().mean(),
        '专业课': group[categories['专业课']].mean().mean(),
        '体育教育': group['体育教育满意度'].mean(),
        '美育教育': group['美育教育满意度'].mean(),
        '劳动教育': group['劳动教育满意度'].mean()
    })
college_means = pd.DataFrame(college_means)

# 3. 为每个参数单独绘制柱状图
parameters = ['思政课', '专业课', '体育教育', '美育教育', '劳动教育']

for param in parameters:
    # 创建新的图形
    plt.figure(figsize=(15, 8))

    # 按当前参数值排序学院
    sorted_colleges = college_means.sort_values(by=param, ascending=False)

    # 绘制柱状图
    bars = plt.bar(sorted_colleges['学院'], sorted_colleges[param], color=plt.cm.tab20(np.arange(len(sorted_colleges))))

    # 添加数值标签
    for bar in bars:
        height = bar.get_height()
        plt.annotate(f'{height:.2f}',
                     xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3),  # 3 points vertical offset
                     textcoords="offset points",
                     ha='center', va='bottom')

    # 设置图形属性
    plt.title(f'各学院{param}满意度对比', fontsize=16)
    plt.xlabel('学院', fontsize=14)
    plt.ylabel('平均满意度', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, 1.2)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()